In [11]:
import tensorflow as tf
import time

In [12]:
tf.__version__

'2.8.0'

In [13]:
class FileDataset(tf.data.Dataset):
    def read_file_in_batches(num_samples):
        # Opening the file
        time.sleep(0.03)

        for sample_idx in range(num_samples):
            # Reading data (line, record) from the file
            time.sleep(0.015)

            yield (sample_idx,)

    def __new__(cls, num_samples=3):
        return tf.data.Dataset.from_generator(
            cls.read_file_in_batches,
            output_signature = tf.TensorSpec(shape = (1,), dtype = tf.int64),
            args=(num_samples,)
        )

In [14]:
def benchmark(dataset, num_epochs=2):
    for epoch_num in range(num_epochs):
        for sample in dataset:
            # Performing a training step
            time.sleep(0.01)

In [17]:
%%timeit
benchmark(FileDataset())

372 ms ± 24.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:
%%timeit
benchmark(FileDataset().prefetch(tf.data.AUTOTUNE))

313 ms ± 19.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
%%timeit
benchmark(FileDataset().prefetch(2))

303 ms ± 11.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [21]:
dataset = tf.data.Dataset.range(5)
for d in dataset:
    print(d.numpy())

0
1
2
3
4


In [22]:
dataset.map(lambda x: x**2)
for d in dataset:
    print(d.numpy())

0
1
2
3
4


In [24]:
dataset = dataset.cache()
list(dataset.as_numpy_iterator())

[0, 1, 2, 3, 4]

In [25]:
list(dataset.as_numpy_iterator())

[0, 1, 2, 3, 4]

In [28]:
def mapped_function(s):
    tf.py_function(lambda: time.sleep(0.03), [], ())
    return s

In [31]:
%%timeit -n1 -r1
benchmark(FileDataset().map(mapped_function), 5)

1.48 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [33]:
%%timeit -n1 -r1
benchmark(FileDataset().map(mapped_function).cache(), 5)

525 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
